# 第六章 评估
 - [二、 创建LLM应用](#二、-创建LLM应用)
     - [2.1 创建评估数据点](#2.1-创建评估数据点)
     - [2.2 创建测试用例数据](#2.2-创建测试用例数据)
     - [2.3 通过LLM生成测试用例](#2.3-通过LLM生成测试用例)
     - [2.4 组合用例数据](#2.4-组合用例数据)
 - [三、 人工评估](#三、-人工评估)
     - [3.1 如何评估新创建的实例](#3.1-如何评估新创建的实例)
     - [3.2 中文版](#3.2-中文版)
 - [四、 通过LLM进行评估实例](#四、-通过LLM进行评估实例)
     - [4.1  评估思路](#4.1--评估思路)
     - [4.2 结果分析](#4.2-结果分析)
     - [3.3 通过LLM进行评估实例](#3.3-通过LLM进行评估实例)


In [4]:
import os, sys
import openai
import langchain
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 


openai.api_base = os.environ["OPENAI_API_BASE"] # 换成代理，一定要加v1
openai.api_key = os.environ["OPENAI_API_KEY"]

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(
    temperature=0.8,
    model_name="gpt-3.5-turbo",
)
chat = ChatOpenAI(
    temperature=0.0,
    model_name = "gpt-3.5-turbo",
)
# os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
# os.environ["WANDB_PROJECT"] = "langchainPlayGround"



## 二、 创建LLM应用
按照langchain链的方式进行构建

In [5]:
from langchain.chains import RetrievalQA #检索QA链，在文档上进行检索
from langchain.chat_models import ChatOpenAI #openai模型
from langchain.document_loaders import CSVLoader #文档加载器，采用csv格式存储
from langchain.indexes import VectorstoreIndexCreator #导入向量存储索引创建器
from langchain.vectorstores import DocArrayInMemorySearch #向量存储

#加载并查看数据
import pandas as pd
test_data = pd.read_csv('./data/OutdoorClothingCatalog_1000.csv',header=None)

print(test_data[:5])

     0                                                  1  \
0  NaN                                               name   
1  0.0                           Women's Campside Oxfords   
2  1.0           Recycled Waterhog Dog Mat, Chevron Weave   
3  2.0  Infant and Toddler Girls' Coastal Chill Swimsu...   
4  3.0         Refresh Swimwear, V-Neck Tankini Contrasts   

                                                   2  
0                                        description  
1  This ultracomfortable lace-to-toe Oxford boast...  
2  Protect your floors from spills and splashing ...  
3  She'll love the bright colors, ruffles and exc...  
4  Whether you're going for a swim or heading out...  


In [7]:
'''
将指定向量存储类,创建完成后，我们将从加载器中调用,通过文档记载器列表加载
'''

#加载数据
file = './data/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(
    file_path=file,
    encoding = "utf-8",
    )
data = loader.load()

vector_storage = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])


#通过指定语言模型、链类型、检索器和我们要打印的详细程度来创建检索QA链
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=vector_storage.vectorstore.as_retriever(), 
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

### 2.1 创建评估数据点
我们需要做的第一件事是真正弄清楚我们想要评估它的一些数据点，我们将介绍几种不同的方法来完成这个任务

1、将自己想出好的数据点作为例子，查看一些数据，然后想出例子问题和答案，以便以后用于评估

### 2.2 创建测试用例数据

因此，我们可以问一个简单的问题，这个舒适的套头衫套装有侧口袋吗？，我们可以通过上面的内容看到，它确实有一些侧口袋，答案为是
对于第二个文档，我们可以看到这件夹克来自某个系列，即down tech系列，答案是down tech系列。

In [9]:
# 看起来第一个文档中有这个套头衫，
# 第二个文档中有这个夹克，
# 从这些细节中，我们可以创建一些例子查询和答案
# 查看这里的一些文档，我们可以对其中发生的事情有所了解

print(data[10])
print(data[11])

examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported." metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 10}
page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of 

### 2.3 通过LLM自动生成测试用例

In [10]:
#导入QA生成链，它将接收文档，并从每个文档中创建一个问题答案对
from langchain.evaluation.qa import QAGenerateChain 



#通过传递chat open AI语言模型来创建这个链
example_gen_chain = QAGenerateChain.from_llm(
    ChatOpenAI(),
    verbose = True,
    )

for item in data[5:10]:
    print(item)
    # print(data[:5])


new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[5:10]]
) #我们可以创建许多例子

print(new_examples) #查看用例数据
print()
print(new_examples[0])
print()
# print(data[0])

page_content=": 5\nname: Smooth Comfort Check Shirt, Slightly Fitted\ndescription: Our men's slightly fitted check shirt is the perfect choice for your wardrobe! Customers love how it fits right out of the dryer. Size & Fit: Slightly Fitted, Relaxed through the chest and sleeve with a slightly slimmer waist. Fabric & Care: 100% cotton poplin, with wrinkle-free performance that won't wash out. Our innovative TrueCool® fabric wicks moisture away from your skin and helps it dry quickly. Additional Features: Traditional styling with a button-down collar and a single patch pocket. Imported." metadata={'source': './data/OutdoorClothingCatalog_1000.csv', 'row': 5}
page_content=': 6\nname: Mens Outdoor Research Apex Gloves\ndescription: These Outdoor Research Arete gloves are perfect for your backcountry adventures. They feature a tough Taslan nylon shell and removable Gore-Tex® nylon fleece liner that is quick-drying and provides complete protection and warmth. The stretchy nylon Lycra elasta

d:\Anaconda3\envs\LLM\lib\site-packages\langchain\chains\llm.py:307: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.
[{'qa_pairs': {'query': 'What is the fabric composition of the Smooth Comfort Check Shirt, Slightly Fitted?', 'answer': 'The Smooth Comfort Check Shirt, Slightly Fitted is made of 100% cotton poplin fabric.'}}, {'qa_pairs': {'query': 'What are the key features of the Mens Outdoor Research Apex Gloves?', 'answer': 'The Mens Outdoor Research Apex Gloves feature a tough Taslan nylon shell, a removable Gore-Tex® nylon fleece liner, stretchy nylon Lycra elastane soft-shell fabric on the back of the hands, an ergonomic design for maximum comfort, a gauntlet closure to seal out snow and cold wind, and a special pocket for inserting a heat pack to keep hands warm. They can be worn alone or together with the liner for maximum warmth.'}}, {'qa_pairs': {'query': "What is the material composition of the Women's Ultraplush Sweats, Relaxed Crewneck?", 'answer': "The Women's Ultraplush Sweats, Relaxed Crewneck is made of 95% cotton and 5% spandex."}}, {'qa_pairs': {'query': 'What i

### 2.4 组合用例数据

In [11]:

for new_example in new_examples:
    # print(type(new_example))
    print(new_example['qa_pairs'])
    examples.append(new_example['qa_pairs'])

print("\n\n\n")
# qa.run(examples[0]["query"])

for example in examples:
    # print(example)
    print(example["answer"])
    qa.run(example["query"])

{'query': 'What is the fabric composition of the Smooth Comfort Check Shirt, Slightly Fitted?', 'answer': 'The Smooth Comfort Check Shirt, Slightly Fitted is made of 100% cotton poplin fabric.'}
{'query': 'What are the key features of the Mens Outdoor Research Apex Gloves?', 'answer': 'The Mens Outdoor Research Apex Gloves feature a tough Taslan nylon shell, a removable Gore-Tex® nylon fleece liner, stretchy nylon Lycra elastane soft-shell fabric on the back of the hands, an ergonomic design for maximum comfort, a gauntlet closure to seal out snow and cold wind, and a special pocket for inserting a heat pack to keep hands warm. They can be worn alone or together with the liner for maximum warmth.'}
{'query': "What is the material composition of the Women's Ultraplush Sweats, Relaxed Crewneck?", 'answer': "The Women's Ultraplush Sweats, Relaxed Crewneck is made of 95% cotton and 5% spandex."}
{'query': 'What is the name of the fleece jacket described in the document?', 'answer': 'Mounta

## 三、 人工评估
现在有了这些示例，但是我们如何评估正在发生的事情呢？
通过运行一个示例通过链，并查看它产生的输出
在这里我们传递一个查询，然后我们得到一个答案。实际上正在发生的事情，进入语言模型的实际提示是什么？   
它检索的文档是什么？   
中间结果是什么？    
仅仅查看最终答案通常不足以了解链中出现了什么问题或可能出现了什么问题

我们可以看到它首先深入到检索QA链中，然后它进入了一些文档链。如上所述，我们正在使用stuff方法，现在我们正在传递这个上下文，可以看到，这个上下文是由我们检索到的不同文档创建的。因此，在进行问答时，当返回错误结果时，通常不是语言模型本身出错了，实际上是检索步骤出错了，仔细查看问题的确切内容和上下文可以帮助调试出错的原因。    
然后，我们可以再向下一级，看看进入语言模型的确切内容，以及 OpenAI 自身，在这里，我们可以看到传递的完整提示，我们有一个系统消息，有所使用的提示的描述，这是问题回答链使用的提示，我们可以看到提示打印出来，使用以下上下文片段回答用户的问题。
如果您不知道答案，只需说您不知道即可，不要试图编造答案。然后我们看到一堆之前插入的上下文，我们还可以看到有关实际返回类型的更多信息。我们不仅仅返回一个答案，还有token的使用情况，可以了解到token数的使用情况


由于这是一个相对简单的链，我们现在可以看到最终的响应，舒适的毛衣套装，条纹款，有侧袋，正在起泡，通过链返回给用户，我们刚刚讲解了如何查看和调试单个输入到该链的情况。




In [12]:
''' 
LingChainDebug工具可以了解运行一个实例通过链中间所经历的步骤
'''
import langchain
langchain.debug = True

qa.run(examples[4]["query"])#重新运行与上面相同的示例，可以看到它开始打印出更多的信息
# 我们需要为所有示例创建预测，关闭调试模式，以便不将所有内容打印到屏幕上
langchain.debug = False

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is the material composition of the Women's Ultraplush Sweats, Relaxed Crewneck?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "What is the material composition of the Women's Ultraplush Sweats, Relaxed Crewneck?",
  "context": ": 7\nname: Women's Ultraplush Sweats, Relaxed Crewneck\ndescription: This crewneck sweatshirt is crafted from soft french terry, making it feel great after a long day. It also looks so good it could be worn around town. Our most generous fit sits farthest from the body and falls at the hip. It's made of 95% cotton and 5% spandex and is a substantial midweight french terry fabric, smooth on the outside and softly looped on the inside. It's designed with shaping for a flattering, feminine

### 3.1 如何评估新创建的实例
与创建它们类似，可以运行链条来处理所有示例，然后查看输出并尝试弄清楚，发生了什么，它是否正确

## 四、 通过LLM进行评估实例

In [13]:
predictions = qa.apply(examples) #为所有不同的示例创建预测


''' 
对预测的结果进行评估，导入QA问题回答，评估链，通过语言模型创建此链
'''
from langchain.evaluation.qa import QAEvalChain #导入QA问题回答，评估链


#通过调用chatGPT进行评估
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)


graded_outputs = eval_chain.evaluate(examples, predictions)#在此链上调用evaluate，进行评估



> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


### 4.1  评估思路
当它面前有整个文档时，它可以生成一个真实的答案，我们将打印出预测的答，当它进行QA链时，使用embedding和向量数据库进行检索时，将其传递到语言模型中，然后尝试猜测预测的答案，我们还将打印出成绩，这也是语言模型生成的。当它要求评估链评估正在发生的事情时，以及它是否正确或不正确。因此，当我们循环遍历所有这些示例并将它们打印出来时，可以详细了解每个示例

In [17]:
#我们将传入示例和预测，得到一堆分级输出，循环遍历它们打印答案
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    # print(graded_outputs[i])
    print("Predicted result: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Do the Cozy Comfort Pullover Set        have side pockets?
Real Answer: Yes
Predicted Answer: Yes, the Cozy Comfort Pullover Set does have side pockets.
Predicted Grade: CORRECT

Example 1:
Question: What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?
Real Answer: The DownTek collection
Predicted Answer: The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.
Predicted Grade: CORRECT

Example 2:
Question: What is the fabric composition of the Smooth Comfort Check Shirt, Slightly Fitted?
Real Answer: The Smooth Comfort Check Shirt, Slightly Fitted is made of 100% cotton poplin fabric.
Predicted Answer: The fabric composition of the Smooth Comfort Check Shirt, Slightly Fitted is 100% cotton poplin.
Predicted Grade: CORRECT

Example 3:
Question: What are the key features of the Mens Outdoor Research Apex Gloves?
Real Answer: The Mens Outdoor Research Apex Gloves feature a tough Taslan nylon shell, a removable Gore-Tex

### 4.2 结果分析
对于每个示例，它看起来都是正确的，让我们看看第一个例子。
这里的问题是，舒适的套头衫套装，有侧口袋吗？真正的答案，我们创建了这个，是肯定的。模型预测的答案是舒适的套头衫套装条纹，确实有侧口袋。因此，我们可以理解这是一个正确的答案。它将其评为正确。    
#### 使用模型评估的优势

你有这些答案，它们是任意的字符串。没有单一的真实字符串是最好的可能答案，有许多不同的变体，只要它们具有相同的语义，它们应该被评为相似。如果使用正则进行精准匹配就会丢失语义信息，到目前为止存在的许多评估指标都不够好。目前最有趣和最受欢迎的之一就是使用语言模型进行评估。